<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/Boston_Housing_Price_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl


In [45]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 16 13:21 kaggle.json


In [46]:
!kaggle competitions download -c 2020aiboston

2020aiboston.zip: Skipping, found more recently modified local copy (use --force to force download)


In [47]:
!unzip 2020aiboston.zip

Archive:  2020aiboston.zip
replace boston_housing_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: boston_housing_test.csv  
replace boston_housing_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: boston_housing_train.csv  
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   


In [0]:
import pandas as pd
import numpy as np

import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

from sklearn import preprocessing

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [0]:
# 학습 파라미터 설정
learning_rate = 0.01
training_epochs = 1000
batch_size = 50
drop_prob = 0.3
Scaler = preprocessing.StandardScaler()

In [0]:
train_data=pd.read_csv('boston_housing_train.csv',header=None, usecols=range(1,15))
test_data=pd.read_csv('boston_housing_test.csv',header=None, usecols=range(1,14))

In [0]:
x_train_data=train_data.loc[:,0:13]
y_train_data=train_data.loc[:,14]

x_train_data=np.array(x_train_data)
y_train_data=np.array(y_train_data)
x_train_data = Scaler.fit_transform(x_train_data)

x_train_data=torch.FloatTensor(x_train_data)
y_train_data=torch.FloatTensor(y_train_data)

In [0]:
train_dataset = torch.utils.data.TensorDataset(x_train_data, y_train_data)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
#linear1 = torch.nn.Linear(13,8,bias=True)
linear1 = torch.nn.Linear(13,10,bias=True)
linear2 = torch.nn.Linear(10,10,bias=True)
linear3 = torch.nn.Linear(10,10,bias=True)
linear4 = torch.nn.Linear(10,10,bias=True)
linear5 = torch.nn.Linear(10,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [177]:
# Random Init => Xavier Init
torch.nn.init.xavier_uniform(linear1.weight)
torch.nn.init.xavier_uniform(linear2.weight)
torch.nn.init.xavier_uniform(linear3.weight)
torch.nn.init.xavier_uniform(linear4.weight)
torch.nn.init.xavier_uniform(linear5.weight)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  


Parameter containing:
tensor([[-0.6590,  0.4447,  0.4442,  0.3085, -0.4568, -0.2980,  0.4058,  0.4914,
          0.6821,  0.3975]], requires_grad=True)

In [0]:
# ======================================
# relu는 맨 마지막 레이어에서 빼는 것이 좋다.
# ======================================
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)

In [0]:
# 손실함수와 최적화 함수
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [180]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0001 cost = 523.085998535
Epoch: 0002 cost = 377.997009277
Epoch: 0003 cost = 313.061798096
Epoch: 0004 cost = 287.147949219
Epoch: 0005 cost = 280.691253662
Epoch: 0006 cost = 242.871261597
Epoch: 0007 cost = 234.156173706
Epoch: 0008 cost = 206.169021606
Epoch: 0009 cost = 176.952056885
Epoch: 0010 cost = 188.912048340
Epoch: 0011 cost = 194.227508545
Epoch: 0012 cost = 179.128906250
Epoch: 0013 cost = 170.715835571
Epoch: 0014 cost = 168.132095337
Epoch: 0015 cost = 157.171630859
Epoch: 0016 cost = 161.900756836
Epoch: 0017 cost = 153.567169189
Epoch: 0018 cost = 144.058197021
Epoch: 0019 cost = 156.737045288
Epoch: 0020 cost = 148.725769043
Epoch: 0021 cost = 150.015670776
Epoch: 0022 cost = 156.097183228
Epoch: 0023 cost = 138.393112183
Epoch: 0024 cost = 138.600738525
Epoch: 0025 cost = 145.580734253
Epoch: 0026 cost = 136.876937866
Epoch: 0027 cost = 148.546783447
Epoch: 0028 cost = 138.803817749
Epoch: 0029 cost = 134.177673340
Epoch: 0030 cost = 132.398223877
Epoch: 003

In [0]:
# Test the model using test sets
with torch.no_grad():
  model.eval()
  x_test_data=test_data.loc[:,:]
  x_test_data=np.array(x_test_data)
  x_test_data = Scaler.transform(x_test_data)
  x_test_data=torch.from_numpy(x_test_data).float().to(device)

  prediction = model(x_test_data)

In [0]:
correct_prediction = prediction.cpu().numpy().reshape(-1,1)

In [183]:
submit=pd.read_csv('sample_submission.csv')
submit

,id,Price
0,0.0,1.0
1,1.0,1.0
2,2.0,1.0
3,3.0,1.0
4,4.0,1.0
...,...,...
162,162.0,1.0
163,163.0,1.0
164,164.0,1.0
165,165.0,1.0


In [184]:
for i in range(len(correct_prediction)):
  submit['Price'][i]=correct_prediction[i].item()

submit

,id,Price
0,0.0,22.764185
1,1.0,22.763489
2,2.0,22.781445
3,3.0,22.573658
4,4.0,22.741428
...,...,...
162,162.0,22.683640
163,163.0,22.691471
164,164.0,22.746899
165,165.0,22.759819


In [186]:
submit.to_csv('result.csv',index=False,header=True)

!kaggle competitions submit -c 2020aiboston -f result.csv -m "14010974_이기택"

100% 3.91k/3.91k [00:02<00:00, 1.80kB/s]
Successfully submitted to 2020.AI.Boston